## 学習データ数及びエポック数がどの程度モデルの精度に影響を与えるか調べる。

### 💬 概要説明

ベースになるモデルはYOLO v8 (x)である。  
まずエポック数「3」で徐々に学習データを増やしていく。  
一連の流れが終わったら、次にエポック数を「6」「9」「12」「15」「18」「21」「24」と増やしていく。

### 👨‍💻 ライブラリのインポート

In [ ]:
# 機械学習ライブラリ
from ultralytics import YOLO

# ユーティリティライブラリ
import os
import json

# データ分析ライブラリ
import pandas as pd

# データ可視化ライブラリ
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# その他のライブラリ
import labo

### 👨‍💻 モデルの作成

In [ ]:
# データセットのリストを作成
data_list = [f'./datasets_path/data_{i}.yaml' for i in range(1, 7)]

In [ ]:
# エポック数を変えながらモデル作りを繰り返す(3,6,9,12,15,18,21,24)
for epoch_size in range(3,25,3):

    # 指定されたエポック数で徐々に学習データを増やしていく
    for i, data in enumerate(data_list, start=1):

        # 以前のモデルを読み込む
        if i == 1:
            model = YOLO('yolov8x.pt')
        else:
            model = YOLO(model_path)

        # 学習 → モデルを保存
        model.train(data=data, epochs=epoch_size)

        # 評価 → 結果を保存
        model.val(save_json=True)

        # 次の追加学習に向けて、今作ったモデルを呼び出すためのパスを更新する
        if i==1:
            model_path = './runs/detect/train/weights/best.pt'
        else:
            model_path = f'./runs/detect/train{i}/weights/best.pt'
    
    # 各学習データ数でのモデルの精度の検証結果（JSON）をval_jsonフォルダに集める
    detect_folder_path = './runs/detect'
    labo.save_val_json(detect_folder_path)

    # どのエポック数で行ったのかわかるようにrunsフォルダの名前を改名する
    os.rename('./runs', f'./runs_epoch_{epoch_size}')

### 💬 フォルダ構成、JSONの中身を確認

##### フォルダ構成(例)  
runs_epoch_3  
-detect  
--val_json  
---predictions_1.json  
---predictions_2.json  
---predictions_3.json  
---predictions_4.json  
---predictions_5.json  
---predictions_6.json  

##### JSONの中身(例)
{
        "image_id": "4_24_00601",
        "category_id": 0,
        "bbox": [
            323.166,
            2687.309,
            115.57,
            126.051
        ],
        "score": 0.95765
    },

### 👨‍💻 検証結果をデータフレーム化

In [ ]:
# 訓練データ数の対応表
train_data_dict = {"1": 20, "2": 40, "3": 60, "4": 80, "5": 100, "6": 124}

In [ ]:
# データフレームの初期化
df = pd.DataFrame(columns=["Epochs", "Training Data", "Score"])

In [ ]:
# ルートディレクトリの指定
root_dir = "."

In [ ]:
# ルートディレクトリ内の各エポックディレクトリに対して
for epoch_dir in os.listdir(root_dir):
    if "runs_epoch_" in epoch_dir:
        epochs = int(epoch_dir.split("_")[-1])  # エポック数の取得
        json_dir = os.path.join(root_dir, epoch_dir, "detect", "val_json")

        # JSONディレクトリ内の各JSONファイルに対して
        for json_file in os.listdir(json_dir):
            if "predictions_" in json_file:
                training_data = train_data_dict[json_file.split("_")[-1].split(".")[0]]  # 訓練データ数の取得

                # JSONファイルの読み込み
                with open(os.path.join(json_dir, json_file), "r") as f:
                    data = json.load(f)

                # スコアの平均値の計算
                score_avg = sum([item["score"] for item in data]) / len(data)

                # データフレームへのデータの追加
                df = df.append({"Epochs": epochs, "Training Data": training_data, "Score": score_avg}, ignore_index=True)

In [ ]:
# データフレームの表示
print(df)

### 👨‍💻 3次元グラフの作成

x軸：エポック数  
y軸：訓練データ数  
z軸：モデルの精度

In [ ]:
# 3Dグラフの作成
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# データのプロット
ax.scatter(df["Epochs"], df["Training Data"], df["Score"])

# 軸ラベルの設定
ax.set_xlabel('Epochs')
ax.set_ylabel('Training Data')
ax.set_zlabel('Score')

In [ ]:
# グラフの保存
plt.savefig(os.path.expanduser("~/Desktop/3d_plot.png"))

In [ ]:
# Notebook内でグラフを表示
plt.show()